In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
from onnx import *
import os
import scipy.io as sio

In [4]:
mat_contents=sio.loadmat('ACASXU_run2a_1_3_batch_2000.mat')
mat_contents2=sio.loadmat('neural_network_information_0.mat')
del mat_contents['__header__']
del mat_contents['__version__']
del mat_contents['__globals__']
del mat_contents2['__header__']
del mat_contents2['__version__']
del mat_contents2['__globals__']
mat_contents

{'Maximum_of_Inputs': array([[6.076000e+04, 3.141593e+00, 3.141593e+00, 1.200000e+03,
         1.200000e+03]]),
 'Minimum_of_Inputs': array([[  0.      ,  -3.141593,  -3.141593, 100.      ,   0.      ]]),
 'W': array([[array([[-2.06269e-01, -4.85561e-02, -1.30061e-02, -3.66034e-01,
         -3.30736e-01],
        [ 9.88522e-01,  1.15803e-03, -2.75160e-05, -2.98118e-04,
         -1.90740e-03],
        [-1.13278e-02, -3.95710e-03,  1.79798e-02,  1.25019e-02,
          1.45017e-03],
        [ 3.18616e-01,  4.45469e-01,  3.74758e-01, -1.07722e-01,
          1.14876e-01],
        [-6.41880e-02,  5.21507e-03, -6.97606e-04, -4.20328e-01,
          1.00533e+00],
        [ 4.15356e-02, -1.00474e-01, -1.77231e-01, -1.41497e+00,
         -2.58510e-01],
        [-3.08707e-01, -1.03980e+00,  1.19547e+00,  1.15655e-01,
          2.50045e-01],
        [ 1.87642e-01,  3.97281e-02,  4.46064e-02, -1.85308e-01,
         -1.23078e+00],
        [-3.31992e-01,  1.01798e+00, -1.03799e+00,  2.33600e-01,
     

In [5]:
W=mat_contents['W'].reshape((-1,1))
layerSizes=mat_contents['layer_sizes']
layerSizes1=mat_contents2['layer_sizes']
b=mat_contents['b'].reshape((-1,1))
W1=mat_contents2['W'].reshape((-1,1))
b1=mat_contents2['b'].reshape((-1,1))
type(W)
type(b)

numpy.ndarray

In [6]:
def createWeightBiasLabels(W):
    weightNames=[]
    biasNames=[]
    weightPrefix="W"
    biasPrefix="B"
    for i in range(0,len(W)):
        weightNames.append(weightPrefix+str((i+1)))
        biasNames.append(biasPrefix+str((i+1)))
    return weightNames, biasNames

In [ ]:

num_iter=2*num_layers-1
nodeList.append(helper.make_node("FC", ["X", "W1", "B1"], ["H1"]))

In [ ]:
print(weightLabels)
biasLabels

In [ ]:
nodeList[0]

In [13]:
def createReluplexOnnx(W,b,layerSizes):
    num_layers=len(W)
    layer_name="FC"
    input_name="X"
    inputSize=layerSizes[0][0]
    outputSize=layerSizes[0][len(W)]
    output_name="Y"
    reluOutput="R"
    output1="O"
    reluOperation="ReLU"
    weightLabels, biasLabels=createWeightBiasLabels(W)
    nodeList=[]
    tensorList=[]
    tensorList.append(helper.make_tensor_value_info(input_name, TensorProto.FLOAT, [int(inputSize),1]))
    outputList=[helper.make_tensor_value_info(output_name, TensorProto.FLOAT, [int(outputSize),1])]
    for i in range(0,num_layers):
        nodeList.append(helper.make_node(layer_name, [input_name, weightLabels[i], biasLabels[i]], [output1+str(i+1)]))
        input_name=output1+str(i+1)
        if(i==num_layers-1):
            continue
        else:
            nodeList.append(helper.make_node(reluOperation, [input_name], [reluOutput+str(i+1)]))
        input_name=reluOutput+str(i+1)
    for i in range(0,num_layers):
        weightMatrix=W[i][0]
        biasMatrix=b[i][0]
        tensorList.append(helper.make_tensor_value_info(weightLabels[i], TensorProto.FLOAT, [weightMatrix.shape[0],weightMatrix.shape[1]]))
        tensorList.append(helper.make_tensor_value_info(biasLabels[i], TensorProto.FLOAT, [biasMatrix.shape[0],biasMatrix.shape[1]]))
    print(nodeList)
    print(tensorList)
    print(outputList)
    graph=helper.make_graph(nodeList,"MLP",tensorList,outputList)
    model_def = helper.make_model(graph, producer_name='Sherlock->Onnx')
    return model_def
    
    

In [14]:
model=createReluplexOnnx(W,b,layerSizes)

[input: "X"
input: "W1"
input: "B1"
output: "O1"
op_type: "FC"
, input: "O1"
output: "R1"
op_type: "ReLU"
, input: "R1"
input: "W2"
input: "B2"
output: "O2"
op_type: "FC"
, input: "O2"
output: "R2"
op_type: "ReLU"
, input: "R2"
input: "W3"
input: "B3"
output: "O3"
op_type: "FC"
, input: "O3"
output: "R3"
op_type: "ReLU"
, input: "R3"
input: "W4"
input: "B4"
output: "O4"
op_type: "FC"
, input: "O4"
output: "R4"
op_type: "ReLU"
, input: "R4"
input: "W5"
input: "B5"
output: "O5"
op_type: "FC"
, input: "O5"
output: "R5"
op_type: "ReLU"
, input: "R5"
input: "W6"
input: "B6"
output: "O6"
op_type: "FC"
, input: "O6"
output: "R6"
op_type: "ReLU"
, input: "R6"
input: "W7"
input: "B7"
output: "O7"
op_type: "FC"
]
[name: "X"
type {
  tensor_type {
    elem_type: FLOAT
    shape {
      dim {
        dim_value: 5
      }
      dim {
        dim_value: 1
      }
    }
  }
}
, name: "W1"
type {
  tensor_type {
    elem_type: FLOAT
    shape {
      dim {
        dim_value: 50
      }
      dim {
   

In [15]:
new_model_path = os.path.join('/Users/Musau/Documents/Research/NeuralNetworkToolParsers/examples', 'reluExample.onnx')

In [16]:
new_model_path

'/Users/Musau/Documents/Research/NeuralNetworkToolParsers/examples/reluExample.onnx'

In [17]:
onnx.save(model, new_model_path)